In [2]:
import pandas as pd

FB_PATH = "../../artifact/core8/core8_06_fallback_decisions.csv"
fb = pd.read_csv(FB_PATH)

print(fb.columns.tolist())
fb.head()

['run_id', 'case_id', 'antibody_id', 'step', 'blocked_rate_window', 'veto_streak', 'action_toggle_rate', 'SoMS_cumsum_window', 'refusal_triggered', 'refusal_reason_code', 'cooldown_ok', 'block_s', 'veto_s', 'tog_s', 'soms_s', 'fallback_score', 'fallback_evaluated', 'fallback_entered', 'fallback_stage', 'fallback_reason_code', 'fallback_rule_id', 'fallback_score.1', 'is_hold', 'is_partial_seal', 'is_refusal', 'is_fallback']


,run_id,case_id,antibody_id,step,blocked_rate_window,veto_streak,action_toggle_rate,SoMS_cumsum_window,refusal_triggered,refusal_reason_code,...,fallback_evaluated,fallback_entered,fallback_stage,fallback_reason_code,fallback_rule_id,fallback_score.1,is_hold,is_partial_seal,is_refusal,is_fallback
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0,0.0,0,0.0,0.0,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.000,True,False,False,False
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,1,0.0,0,0.0,0.0,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.000,True,False,False,False
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,2,0.0,0,0.0,0.0,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.000,True,False,False,False
3,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,3,0.0,0,0.0,0.0,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.000,True,False,False,False
4,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,4,0.0,0,0.0,0.6,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.006,True,False,False,False


In [4]:
import pandas as pd

FB_PATH = "../../artifact/core8/core8_06_fallback_decisions.csv"
fb = pd.read_csv(FB_PATH)

# ---- score 컬럼 확정 (fallback_score.1 같은 중복 방지) ----
score_col = "fallback_score"
if "fallback_score.1" in fb.columns:
    # 둘 중 하나만 쓰자: 기본은 fallback_score
    pass

# ---- 이벤트 정의 ----
fb["event_hard"] = (fb["refusal_triggered"] == True) | (fb["fallback_stage"] == "REFUSAL") | (fb["is_refusal"] == True)
fb["event_soft"] = (fb["fallback_stage"] == "PARTIAL_SEAL") | (fb["is_partial_seal"] == True)

hard = fb.loc[fb["event_hard"], score_col].dropna()
soft = fb.loc[fb["event_soft"], score_col].dropna()

print("hard event rows:", len(hard))
print("soft event rows:", len(soft))

assert len(hard) + len(soft) > 0, "❌ event(REFUSAL/PARTIAL_SEAL) 기반 데이터도 없음"

# ---- threshold 후보 ----
# hard가 있으면 hard 기준, 없으면 soft 기준으로 잡는다.
base = hard if len(hard) > 0 else soft
desc = base.describe()
print(desc)

threshold_candidates = {
    "aggressive": float(base.quantile(0.10)),   # 더 빨리 터지게
    "balanced":   float(base.quantile(0.25)),   # 데모용
    "conservative": float(base.quantile(0.50)), # 보수적
}

threshold_candidates

hard event rows: 0
soft event rows: 7
count    7.000000
mean     0.483372
std      0.026148
min      0.441709
25%      0.471907
50%      0.477771
75%      0.499521
max      0.521271
Name: fallback_score, dtype: float64


{'aggressive': 0.4585460317460317,
 'balanced': 0.4719065255731922,
 'conservative': 0.4777707231040565}

In [5]:
import pandas as pd

FB_PATH = "../../artifact/core8/core8_06_fallback_decisions.csv"
fb = pd.read_csv(FB_PATH)

score_col = "fallback_score"

# HOLD 구간 제외 (MIN_STEPS 이전은 의미가 다름)
fb2 = fb[fb["fallback_stage"] != "HOLD"].copy()

# 궤적 단위 최대 score
mx = (
    fb2.groupby(["run_id","case_id","antibody_id"], as_index=False)[score_col]
       .max()
       .rename(columns={score_col: "max_fallback_score"})
)

print(mx["max_fallback_score"].describe())

threshold_candidates = {
    "aggressive": float(mx["max_fallback_score"].quantile(0.60)),
    "balanced": float(mx["max_fallback_score"].quantile(0.75)),
    "conservative": float(mx["max_fallback_score"].quantile(0.90)),
}
threshold_candidates

count    6.000000
mean     0.312590
std      0.145027
min      0.140500
25%      0.204000
50%      0.311193
75%      0.395135
max      0.521271
Name: max_fallback_score, dtype: float64


{'aggressive': 0.3808853615520282,
 'balanced': 0.3951353615520282,
 'conservative': 0.46057804232804234}

In [6]:
import pandas as pd

FB_PATH = "../../artifact/core8/core8_06_fallback_decisions.csv"
fb = pd.read_csv(FB_PATH)

soft_events = fb[
    (fb["fallback_evaluated"] == True)
    & (fb["fallback_entered"] == False)
]

assert len(soft_events) > 0, "❌ soft fallback event도 없음"

In [7]:
desc = soft_events["fallback_score"].describe()
print(desc)

count    180.000000
mean       0.139514
std        0.140677
min        0.000000
25%        0.016500
50%        0.093250
75%        0.231368
max        0.521271
Name: fallback_score, dtype: float64


In [8]:
threshold_candidates = {
    # 거의 모든 soft-risk에서 fallback (stress test용)
    "aggressive": float(desc["25%"]),

    # Core11 데모 기본값 (권장)
    "balanced": float(desc["50%"]),

    # 매우 보수적 (fallback 거의 안 남)
    "conservative": float(desc["75%"]),
}

threshold_candidates

{'aggressive': 0.0165,
 'balanced': 0.09325,
 'conservative': 0.23136772486772483}